## Importation des modules

In [1]:
!pip install wikipedia-api
!pip install pyvis
!pip install eventlet

  Using cached pyvis-0.3.1.tar.gz (748 kB)
  Created wheel for pyvis: filename=pyvis-0.3.1-py3-none-any.whl size=755833 sha256=3d95652cb490fe1725adcc36a6b08fec9e6b09d0a4560992b9a709ba3617be59
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\a4\0c\61\8469ca276f96ab772c3acc7f47d71e9737cbdf6f446f017f48
Successfully built pyvis
  Using cached eventlet-0.33.2-py2.py3-none-any.whl (226 kB)
  Using cached dnspython-2.2.1-py3-none-any.whl (269 kB)


In [2]:
import wikipediaapi
import pandas as pd
import numpy as np

## Création d'une fonction qui recherche les pages voisines d'une page wikipédia

In [3]:
# On ne considère que le Wikipedia en français

wiki_wiki = wikipediaapi.Wikipedia("fr", extract_format=wikipediaapi.ExtractFormat.WIKI)

In [4]:
# On va commencer par écrire une fonction qui retourne la liste des voisins d'une page donnée

def get_neighbors(name):
    
    page = wiki_wiki.page(name)
    return list(page.links.keys())

In [5]:
# On doit ensuite trouver les voisins des voisins, les ajouter dans une pile de pages à parcourir, puis trouver leurs voisins et recommencer, 
# et ainsi de suite jusqu'à ce que la pile soit vide (l'élément d'indice 0 est le fond de la pile)
# On parle alors de parcours en profondeur du graphe (qu'on peut voir comme un arbre ici)
# On va représenter le graphe sous forme d'un dictionnaire d'adjacence, de la forme {"sommet" : [voisins]}

# Première version pour évaluer le type des liens présents dans des pages quelconques :

def get_tree(start):
    i = 0
    pile = [start]
    tree = {}
    while pile != [] and i < 50:
        node = pile.pop()
        i += 1
        print(node) # On va visualiser les noms de pages qu'on traite
        tree[node] = []
        for neighbor in get_neighbors(node):
            tree[node].append(neighbor)
            if neighbor not in tree.keys():
                pile.append(neighbor)
    return tree

In [ ]:
# On fait un appel à la fonction get_tree pour regarder ce qu'il se passe

get_tree("Python_(langage)")

In [6]:
# Problème : On voit qu'il y a beaucoup de liens qui mènent vers des pages "indésirables", comme les modèles, 
# les discussions, les aides etc

# On va donc réaliser un filtrage d'expression régulière sur le nom des pages, à l'aide du module re :

import re

def filtering(name):
    if re.match(r"Portail:.*", name):
        return False
    if re.match(r"Discussion Projet:.*", name):
        return False
    if re.match(r"Discussion:.*", name):
        return False
    if re.match(r"Module:.*", name):
        return False
    if re.match(r"Projet:.*", name):
        return False
    if re.match(r"Modèle:.*", name):
        return False
    if re.match(r"Aide:.*", name):
        return False
    if re.match(r"Catégorie:.*", name):
        return False
    if re.match(r"Utilisateur:.*", name):
        return False
    if re.match(r"Utilisatrice:.*", name):
        return False
    if re.match(r"Discussion Utilisateur:.*", name):
        return False
    if re.match(r"Sujet:.*", name):
        return False
    if re.match(r"Référence:.*", name):
        return False
    if re.match(r"Discussion Wikipédia:.*", name):
        return False
    if re.match(r"Fichier:.*", name):
        return False
    if re.match(r"Discussion modèle:.*", name):
        return False
    if re.match(r"Discussion Portail:.*", name):
        return False
    if re.match(r"Discussion catégorie:.*", name):
        return False
    if re.match(r"Wikipédia:.*", name):
        return False
    if re.match(r"Discussion utilisateur:.*", name):
        return False
    return True

## Fonction d'appel API pour récupérer des pages wikipédia

In [9]:
# On rajoute une ligne dans la fonction :

import numpy as np # On a besoin des fonction numpy sur les tableaux

def get_10000_nodes(pile, tree):
    i = 0
    while pile != [] and i<10000:
        global node
        node = pile.pop()
        if node not in tree.keys():
            # print(node)
            tree[node] = [[]]
            i += 1
            neighbors = get_neighbors(node)
            if len(neighbors)>0:
                for neighbor in neighbors:
                    if filtering(neighbor) and len(neighbor)>1:
                        tree[node].append(neighbor)
                        if neighbor not in tree.keys():
                            pile.append(neighbor)
    return list(np.unique(np.array(pile))), tree
import requests
import eventlet # Pour réduire la possibilité de timeout à la lecture

global pile, tree # Pour pouvoir garder la progression en cas d'erreur, seule la page d'erreur est perdue, ce n'est pas très grave

def get_tree_clean(start = "Python_(langage)", pile = [], tree = {}): #On cherche tout le réseau de pages, en partant de la page de Python
    j = 0 # Simple compteur pour savoir la vitesse approximative en exécution
    with eventlet.Timeout(40000): #Pour empêcher les timeouts
        while pile != []: # On traite tous les sommets possibles
            pile, tree = get_10000_nodes(pile, tree) # On actualise la valeur toutes les 10 000 pages 
            j += 1
            print(j)

    return tree

## Visualisation du réseau de pages Wikipédia

In [26]:
# Pour visualiser le réseau des pages Wikipédia, on va devoir transformer le dictionnaire d'adjacence en dataframe pandas
# Le dataframe comporte 4 colonnes : "Source", "Target", "Type" et "Weight"
# Le graphe étant orienté, le type d'arête est "Directed"

def create_dataframe(tree, nb_nodes):
    data = {"Source" : [], "Target" : [], "Size" : [], "Weight" : []}
    i = 0
    for node in tree.keys():
        i+=1
        if i == nb_nodes:
            return pd.DataFrame(data)
        else:
            for neighbor in tree[node]:
                if neighbor in tree.keys():
                    weight = len(tree[neighbor])
                else:
                    weight = 0
                data["Source"].append(node)
                data["Target"].append(neighbor)
                data["Size"].append(len(tree[node]))
                data["Weight"].append(weight)
    return pd.DataFrame(data)

In [57]:
# On visualise alors le graphe, en utilisant le module PyVis :

from pyvis.network import Network

def visualize(data, c):
    net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", notebook = True)
    net.barnes_hut()
    sources = data["Source"]
    targets = data["Target"]
    weights = data["Weight"]
    sizes = data["Size"]

    edge_data = zip(sources, targets, weights, sizes)

    for e in edge_data:
        src = e[0]
        dst = e[1]
        w = e[2]
        sz = e[3]

        net.add_node(src, src, size = sz/10, title = src)
        net.add_node(dst, dst, size = w/10, stitle = dst)
        net.add_edge(src, dst, value = w**2)
    for i in range(len(c)-1):
        src = c[i]
        dst = c[i+1]
        net.add_node(src, src, size = 400, title = src)
        net.add_node(dst, dst, size = 400, title = dst)
        net.add_edge(src, dst, width = 4000, color = "red" )
    net.show("Wiki_map.html")

In [3]:
tree = pd.read_pickle("https://minio.lab.sspcloud.fr/cheitz/tree.pickle") # On charge la base de données enregistrée au format pickle

In [147]:
data = create_dataframe(tree, 100000) # On ne crée que les 100 000 premières arêtes, pour des raisons de stockage mémoire

In [174]:
data2 = data[:50000] # On se limite à un plus petit nombre d'arêtes pour pouvoir visualiser le graphe

In [175]:
visualize(data2, []) # On crée le fichier HTML qui code le graphe

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


## Insertion de l'algorithme de Dijkstra

In [4]:
def initialize(tree, deb): # Fonction d'initialisation de l'algorithme de Dijkstra
    d = {} # Dictionnaire de la forme {"sommet" : (distance au sommet de départ, prédecesseur dans le chemin vers le sommet d'arrivée)}
    sommets = list(tree.keys())
    for v in tree.values():
        sommets += v
    sommets = list(np.unique(sommets))
    for s in sommets:
        d[s] = (np.inf, None)
    d[deb] = (0, None)
    return d

In [5]:
def find_min(d, Q): # Fonction pour trouver le sommet de distance minimale au sommet de départ dans un sous-graphe Q du graphe de départ
    mini = np.inf
    sommet = None
    for s in Q.keys():
        if d[s][0]<mini:
            mini = d[s][0]
            sommet = s
    return sommet

In [6]:
def maj_distances(d, s1, s2): # Fonction de mise à jour des distances pour 2 sommets donnés
    if d[s2][0]>d[s1][0]+1:
        d[s2] = (d[s1][0]+1,s1)

In [7]:
def dijkstra(tree, deb, fin): # Algorithme de Dijkstra
    d = initialize(tree, deb)
    Q = tree.copy()
    s1 = None
    while Q!={} and s1!=fin:
        s1 = find_min(d, Q)
        if s1 == None:
            return None
        Q.pop(s1)
        for s2 in tree[s1]:
            if s2 in d.keys():
                maj_distances(d, s1, s2)
    return d # On renvoie le dictionnaire {"sommet" : (distance, prédecesseur)}

In [8]:
def chemin(tree, deb, fin): # Fonction qui permet de récupérer le chemin le plus court entre le départ et l'arrivée
    d = dijkstra(tree, deb, fin)
    if d == None:
        print("Pas de chemin trouvé entre vos 2 sommets :( ")
        pass
    else:
        A = []
        s = fin
        while s!=deb:
            A = [s]+A
            s = d[s][1]
        return [deb]+A

In [10]:
nodes = list(tree.keys()) # On prend la liste de tous les sommets du graphe
len(nodes)

605470

### Attention, le choix de 10 000 sommets est arbitraire, et correspond aux caractéristiques de mon ordinateur !!

In [11]:
nodes = nodes[:10000] # On doit se limiter aux 10 000 premiers sommets pour des raisons de tractabilité 
tree_10k = {k : tree[k] for k in nodes}

In [15]:
deb, fin = np.random.choice(nodes), np.random.choice(nodes) # On choisit un début et une fin aléatoirement
print(deb, fin)

Óscar Iván Zuluaga Élections législatives cubaines de 1956


In [37]:
c = chemin(tree_10k, deb, fin) # On regarde quel est le chemin le plus court entre deb et fin
print(c)

['Óscar Iván Zuluaga', 'Équateur (pays)', "État d'exception", "État d'urgence en France", 'Élection présidentielle française de 2017', 'Élections législatives et présidentielles dans le monde', 'Élections législatives cubaines de 2018', 'Élections législatives cubaines de 1956']


In [59]:
nodes_10k = list(tree_10k.keys())
subgraph = []
while not all(i in subgraph for i in c):
    subgraph = np.random.choice(nodes_10k, 20000)
subtree = {k : tree_10k[k] for k in nodes_10k}
subdata = create_dataframe(subtree, 100000)

In [ ]:
visualize(subdata, c) # On visualise le chemin qu'on a obtenu de la façon suivante :

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


## Création d'une interface graphique

<span style="color:red"> Attention, l'ensemble du code qui suit est à faire tourner tel quel dans un IDE, car il nécessite d'ouvrir une fenêtre tierce !! </span> <br>
Nous vous conseillons d'utiliser "Python_(langage)" et "États-Unis" comme pages de départ et d'arrivée, car nous nous sommes restreints à un petit nombre de sommets; sinon, vous pouvez en tirer 2 au hasard parmi nodes_10k avec la fonction get_random

In [ ]:
import pandas as pd
import numpy as np
from tkinter import *
import tkinter.font as font

tree = pd.read_pickle("https://minio.lab.sspcloud.fr/cheitz/tree.pickle") # On charge la base de données enregistrée au format pickle
nodes = list(tree.keys()) # On prend la liste de tous les sommets du graphe
nodes_10k = nodes[:10000] # On doit se limiter aux 10 000 premiers sommets pour des raisons de tractabilité 
tree_10k = {k : tree[k] for k in nodes_10k}

def get_random():
    return np.random.choice(nodes_10k)

depart = get_random()
arrivee = get_random()

def initialize(tree, deb): # Fonction d'initialisation de l'algorithme de Dijkstra
    d = {} # Dictionnaire de la forme {"sommet" : (distance au sommet de départ, prédecesseur dans le chemin vers le sommet d'arrivée)}
    sommets = list(tree.keys())
    for v in tree.values():
        sommets += v
    sommets = list(np.unique(sommets))
    for s in sommets:
        d[s] = (np.inf, None)
    d[deb] = (0, None)
    return d

def find_min(d, Q): # Fonction pour trouver le sommet de distance minimale au sommet de départ dans un sous-graphe Q du graphe de départ
    mini = np.inf
    sommet = None
    for s in Q.keys():
        if d[s][0]<mini:
            mini = d[s][0]
            sommet = s
    return sommet

def maj_distances(d, s1, s2): # Fonction de mise à jour des distances pour 2 sommets donnés
    if d[s2][0]>d[s1][0]+1:
        d[s2] = (d[s1][0]+1,s1)

def dijkstra(tree, deb, fin): # Algorithme de Dijkstra
    d = initialize(tree, deb)
    Q = tree.copy()
    s1 = None
    while Q!={} and s1!=fin:
        s1 = find_min(d, Q)
        if s1 == None:
            return None
        Q.pop(s1)
        for s2 in tree[s1]:
            if s2 in d.keys():
                maj_distances(d, s1, s2)
    return d # On renvoie le dictionnaire {"sommet" : (distance, prédecesseur)}

def chemin(tree, deb, fin): # Fonction qui permet de récupérer le chemin le plus court entre le départ et l'arrivée
    d = dijkstra(tree, deb, fin)
    if d == None:
        print("Pas de chemin trouvé entre vos 2 sommets :( ")
        pass
    else:
        A = []
        s = fin
        while s!=deb:
            A = [s]+A
            s = d[s][1]
        return [deb]+A

def print_chemin(): # Fonction qui sert à effectuer la recherche de plus court chemin dans l'interface graphique

    depart = fenêtre.winfo_children()[1].get()
    arrivee = fenêtre.winfo_children()[2].get()

    if depart not in nodes_10k:
        result = depart + " n'est pas dans la liste des pages traitées."
    elif arrivee not in nodes_10k:
        result = arrivee + " n'est pas dans la liste des pages traitées."
    else:
        c = chemin(tree_10k, depart, arrivee)
        result = "Le plus court chemin est : " + str(c)

    reset_widgets(depart, arrivee, result)

#Création de la fenêtre et de son titre
fenêtre=Tk()
fenêtre.title("Plus court chemin Wikipedia")
fenêtre.config(bg = "blue") #couleur de l'arrière-plan
fenêtre.geometry("800x600") #dimensions de la fenêtre


def reset_widgets(s1 = "page de départ, ex : Python(langage)", s2 = "page d'arrivée, ex : Chocolat", result = "Veuillez effectuer une recherche."):

    for w in fenêtre.winfo_children():
        w.destroy()

    #Création d'un label pour la saisie des sommets à relier
    SommetLabel = Label(fenêtre, text="Saisir les pages wikipedia de départ et d'arrivée : ", bg = "blue", fg='white')
    SommetLabel['font'] = font.Font(family='Times New Roman', size=18, weight="bold")
    SommetLabel.pack(pady=10)

    #Création des zones de saisie
    sommet1 = StringVar()
    sommet2 = StringVar()

    sommet1.set(s1)
    sommet2.set(s2)

    saisieSommet1 = Entry(fenêtre, textvariable=sommet1, width=35)
    saisieSommet2 = Entry(fenêtre, textvariable=sommet2, width=35)

    saisieSommet1['font'] = font.Font(family='Times New Roman', size=15)
    saisieSommet2['font'] = font.Font(family='Times New Roman', size=15)

    saisieSommet1.pack(pady=10)
    saisieSommet2.pack(pady=5)

    #Création d'un bouton

    command_button = Button(fenêtre, text="CALCULER CHEMIN", relief=GROOVE, width=17, height=1, bg="yellow", command=lambda: print_chemin())
    command_button['font'] = font.Font(family='Times New Roman', size=15, weight="bold")
    command_button.pack(pady=20)

    ResultatLabel = Label(fenêtre, text=result, bg = "blue", fg='white')
    ResultatLabel['font'] = font.Font(family='Times New Roman', size=18, weight="bold")
    ResultatLabel.pack(pady=100)

reset_widgets()
fenêtre.mainloop()

In [ ]:
## 2ème version de l'interface graphique (plus moderne)

In [ ]:
import pandas as pd
import numpy as np
import customtkinter


tree = pd.read_pickle("https://minio.lab.sspcloud.fr/cheitz/tree.pickle") # On charge la base de données enregistrée au format pickle
nodes = list(tree.keys()) # On prend la liste de tous les sommets du graphe
nodes_10k = nodes[:10000] # On doit se limiter aux 10 000 premiers sommets pour des raisons de tractabilité 
tree_10k = {k : tree[k] for k in nodes_10k}

def get_random():
    return np.random.choice(nodes_10k)

depart = get_random()
arrivee = get_random()

def initialize(tree, deb): # Fonction d'initialisation de l'algorithme de Dijkstra
    d = {} # Dictionnaire de la forme {"sommet" : (distance au sommet de départ, prédecesseur dans le chemin vers le sommet d'arrivée)}
    sommets = list(tree.keys())
    for v in tree.values():
        sommets += v
    sommets = list(np.unique(sommets))
    for s in sommets:
        d[s] = (np.inf, None)
    d[deb] = (0, None)
    return d

def find_min(d, Q): # Fonction pour trouver le sommet de distance minimale au sommet de départ dans un sous-graphe Q du graphe de départ
    mini = np.inf
    sommet = None
    for s in Q.keys():
        if d[s][0]<mini:
            mini = d[s][0]
            sommet = s
    return sommet

def maj_distances(d, s1, s2): # Fonction de mise à jour des distances pour 2 sommets donnés
    if d[s2][0]>d[s1][0]+1:
        d[s2] = (d[s1][0]+1,s1)

def dijkstra(tree, deb, fin): # Algorithme de Dijkstra
    d = initialize(tree, deb)
    Q = tree.copy()
    s1 = None
    while Q!={} and s1!=fin:
        s1 = find_min(d, Q)
        if s1 == None:
            return None
        Q.pop(s1)
        for s2 in tree[s1]:
            if s2 in d.keys():
                maj_distances(d, s1, s2)
    return d # On renvoie le dictionnaire {"sommet" : (distance, prédecesseur)}

def chemin(tree, deb, fin): # Fonction qui permet de récupérer le chemin le plus court entre le départ et l'arrivée
    d = dijkstra(tree, deb, fin)
    if d == None:
        print("Pas de chemin trouvé entre vos 2 sommets :( ")
        pass
    else:
        A = []
        s = fin
        while s!=deb:
            A = [s]+A
            s = d[s][1]
        return [deb]+A

def print_chemin(): # Fonction qui sert à effectuer la recherche de plus court chemin dans l'interface graphique

    depart = frame.winfo_children()[1].get()
    arrivee = frame.winfo_children()[2].get()

    if depart not in nodes_10k:
        result = depart + " n'est pas dans la liste des pages traitées."
    elif arrivee not in nodes_10k:
        result = arrivee + " n'est pas dans la liste des pages traitées."
    else:
        c = chemin(tree_10k, depart, arrivee)
        result = "Le plus court chemin est : " + str(c)

    reset_widgets(depart, arrivee, result)

#Création de la fenêtre et de son titre

customtkinter.set_appearance_mode("light")
customtkinter.set_default_color_theme("green")

fenetre=customtkinter.CTk()
fenetre.title("Plus court chemin Wikipedia")
fenetre.geometry("500x350") # dimensions de la fenêtre
frame = customtkinter.CTkFrame(master=fenetre)
frame.pack(pady=20, padx=60, fill="both", expand=True)


def reset_widgets(s1 = "page de départ, ex : Python(langage)", s2 = "page d'arrivée, ex : Chocolat", result = "Veuillez effectuer une recherche."):

    for w in frame.winfo_children():
        w.destroy()

    
    
    #Création d'un label pour la saisie des sommets à relier
    label = customtkinter.CTkLabel(master=frame, text="Calcul du plus court chemin", font=("Roboto", 24))
    label.pack(pady=12, padx=10)

    #Création des zones de saisie
    sommet1 = customtkinter.CTkEntry(master=frame, placeholder_text=s1)
    sommet1.pack(pady=12, padx=10)
    sommet2 = customtkinter.CTkEntry(master=frame, placeholder_text=s2)
    sommet2.pack(pady=12, padx=10)

    #Création d'un bouton
    button = customtkinter.CTkButton(master=frame, text="calculer", command=lambda: print_chemin())
    button.pack(pady=12,padx=10)

    #Création du label avec le résultat
    ResultatLabel = customtkinter.CTkLabel(master=frame, text=result, font=("Roboto", 12))
    ResultatLabel.pack(pady=12, padx=10)

reset_widgets()
fenetre.mainloop()